In [21]:
import os, sys
import glob
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4 as nc
import wrf
import datetime
import xoak
import cartopy.crs as ccrs
# import personal modules
# Path to modules
sys.path.append('../../modules')

# Import my modules
from wrf_funcs_preprocess import select_single_coord_WRF
from ar_funcs import get_ar_days, duration_stats
from timeseries import select_months_ds, select_months_df
from utils import find_perpindicular_line, find_parallel_line, find_intersection_two_lines

In [2]:
# Set up paths

path_to_data = '/home/sbarc/students/nash/data/'
path_to_out  = '../../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../../figs/'      # figures

## Get coordinates of locations needed for WVF climatology

In [3]:
## Get lines and points for mesoscale analysis

# hlat, hlon, tlat, tlon
Line1 = [39.5, 71., 32.25, 81.]
x1, y1, x2, y2 = Line1[3],Line1[2],Line1[1],Line1[0]

# calculate parallel lines
Line2, eq1, eq2 = find_parallel_line(x1, y1, x2, y2, -3., 69.,  80.)
Line3, eq1, eq3 = find_parallel_line(x1, y1, x2, y2, -2., 69.,  80.)

## calculate perpindicular cross section lines
newx_lst = [74.5, 76., 76.25, 78.5]
newx_lst = [74.25, 76.25]
newline = []
ptlst = []
for i, newx in enumerate(newx_lst):
    # get perpindicular line
    line, eq = find_perpindicular_line(x1, y1, x2, y2, newx)
    newline.append(line)
    # get intersecting point for each newline and Line3
    pt = find_intersection_two_lines(eq2[0], eq2[1], eq[0], eq[1])
    ptlst.append(pt)
# print(ptlst)

## add points from radiosonde locations
# ptlst.append([71.38, 42.85]) # Station 38341
# ptlst.append([77.20, 28.58]) # station 42182
ptlst.append([74.0225, 34.0876]) # Feb2010 Landslide Loc
ptlst.append([72.6625, 34.8733]) # Feb2010 Landslide Loc


ptlst = ptlst[2:] ## already processed first two points
print(ptlst)


[[74.0225, 34.0876], [72.6625, 34.8733]]


## Get list of AR dates for specified lat/lon

In [4]:
def ar_dates_single_coord(start_date, end_date, lat1, lon1, mons, mone):
    filename =  'globalARcatalog_ERA-Interim_1979-2019_v3.0.nc'

    # open ds
    ds = xr.open_dataset(path_to_data + 'ar_catalog/' + filename, chunks={'time': 1460}, engine='netcdf4')
    ds = ds.squeeze()
    # remove lev and ens coords
    ds = ds.reset_coords(names=['lev', 'ens'], drop=True)

    
    # select dates within start_date, end_date and months
    ds = ds.sel(time=slice(start_date, end_date))
    ## select only months we are interested in
    ds = select_months_ds(ds, mons, mone)
    
    ## select single coordinate
    ds = ds.sel(lat=lat1, lon=lon1, method="nearest")


    
    ## now get list of dates where ds.shape != np.nan
    # convert dataset to dataframe
    df = ds.shape.to_dataframe(dim_order=['time'])
    df = df.dropna(axis='rows')
    
    dates = df.index.values
    
    return df

,AR_CAT
date,
1979-12-02,1
1979-12-09,1
1979-12-16,1
1979-12-21,1
1979-12-22,1
...,...
2014-12-24,1
2015-01-19,1
2015-01-20,1


In [ ]:
start_date = '1980-01-01'
end_date = '2015-02-28'
mons = 1
mone = 2
domain ='d02'

dates = []
for i, pt in enumerate(ptlst):
    times = ar_dates_single_coord(start_date, end_date, pt[1], pt[0], mons, mone)
    dates.append(times)

In [ ]:
def wrf_clim_filenames(year, domain):
        
    if year == 2015:
        # get list of filenames that contain data from that year from current year folder
        filenames = []
        for name in glob.glob('/home/hasia/' + str(year-1) + '/wrfout_{0}_*'.format(domain)):
            filenames.append(name)
        # sort filenames so they are in chronological order
        filenames = sorted(filenames)
        
        # just get last 18 files
        filenames = filenames[-18:-5]
        
    else:
        # get list of filenames that contain data from that year from previous year folder
        filenames = []
        for name in glob.glob('/home/hasia/' + str(year-1) + '/wrfout_{0}_*'.format(domain)):
            filenames.append(name)
        # sort filenames so they are in chronological order
        filenames = sorted(filenames)
        # only get last 18 files
        filenames = filenames[-18:-5]
        
    return filenames

In [5]:
def wrf_clim_filenames(year, domain):
    filenames = []

    # get list of December filenames that contain data from current year folder
    for name in glob.glob('/home/hasia/{0}/wrfout_{1}_{0}-12*'.format(str(year), domain)):
        filenames.append(name)
        
    # get list of Jan filenames that contain data from current year folder
    for name in glob.glob('/home/hasia/{0}/wrfout_{1}_{2}-01*'.format(str(year), domain, str(year+1))):
        filenames.append(name)
    # get list of Feb filenames that contain data from current year folder
    for name in glob.glob('/home/hasia/{0}/wrfout_{1}_{2}-02*'.format(str(year), domain, str(year+1))):
        filenames.append(name)

    # sort filenames so they are in chronological order
    filenames = sorted(filenames)
    
    return filenames

In [40]:
# wrfin = nc.Dataset(fnames[0])


# ## get the index of the lat/lon values
# slon, slat = ptlst[0]
# x_y = wrf.ll_to_xy(wrfin, slat, slon, timeidx=0, squeeze=True, meta=True, stagger=None, as_int=True)
# x = x_y.values[0] # index of lon value
# y = x_y.values[1] # index of lat value

# ## Get a 1D vertical column for pressure at location x,y
# p_1d = wrf.getvar(wrfin, "pres", units="hPa")[:,y,x]

# ## Get a 1D vertical column for uwind at location x,y
# ua_1d = wrf.getvar(wrfnc, "ua", units="m s-1")[:,y,x]

# levels = np.asarray([750., 700., 650., 600., 550., 500., 450., 400., 350., 300.,250.])

# ## interpolate the values to the requested pressure levels
# interp_vals = wrf.interp1d(ua_1d, p_1d, levels)
# interp_vals

<xarray.DataArray 'ua_z' (z: 11)>
array([-5.2920356 , -0.18566966,  3.2787414 ,  7.3345227 , 12.052273  ,
       18.514317  , 25.677513  , 32.77365   , 39.57352   , 49.01052   ,
       64.82248   ], dtype=float32)
Coordinates:
  * z        (z) float64 750.0 700.0 650.0 600.0 ... 400.0 350.0 300.0 250.0
Attributes:
    description:    destaggered u-wind component
    units:          m s-1
    _FillValue:     None
    missing_value:  None

## Now get the WRF wvf values on those datetimes

In [7]:
%%time
varlst = ('pressure', 'ua', 'va', 'z', 'QVAPOR', 'HGT')
## for each list of dates, 
yr_lst = np.arange(1979, 2015, 1)
for j, pt in enumerate(ptlst):
    lat = pt[1]
    lon = pt[0]
    ds_lst = []
    for i, yr in enumerate(yr_lst):
        # get the list of dates and times within that year
        idx = (ar_cat.index >= '{0}-12-01'.format(str(yr))) & (ar_cat.index < '{0}-03-01'.format(str(yr+1)))
        timesteps = ar_cat.loc[idx].index.values
        # now get the wrf filenames for that year between jan/feb
        wrf_filenames = wrf_clim_filenames(yr, domain)

        if len(timesteps) == 0:
            print('No ARs during', yr)
        else:
            # now we can use this function to search the given wrf files for those timesteps and process the vertical values for the given location
            ds = select_single_coord_WRF(wrf_filenames, varlst, lat, lon, timesteps)

            # calculate specific humidity from mixing ratio (units: kg kg-1)
            ds = ds.assign(q=lambda ds: np.divide(ds.QVAPOR, (1+ds.QVAPOR)))

            # compute vertical moisture flux (units: m s-1*kg kg-1)
            uq = ds.ua*ds.q
            vq = ds.va*ds.q
            ds = ds.assign(wvf=lambda ds: np.sqrt(uq**2 + vq**2))
            ds_lst.append(ds)
    
    # concat/merge ds_lst to single ds file
    ds_final = xr.concat(ds_lst, dim='Time')
    
    # write to netCDF
    varname = 'wvflux'
    latstr = '{:.2f}N'.format(lat)
    lonstr = '{:.2f}E'.format(lon)
    fname = path.join(path_to_data, 'wrf_hasia/{0}/{1}_{2}_{3}.nc').format(domain, varname, latstr, lonstr)
    ds_final.to_netcdf(path=fname, mode = 'w', format='NETCDF4')


/home/hasia/1979/wrfout_d02_1980-01-30_03:00:00 timeidx is  25
/home/hasia/1979/wrfout_d02_1980-01-30_03:00:00 timeidx is  37
/home/hasia/1979/wrfout_d02_1980-02-09_03:00:00 timeidx is  29
/home/hasia/1979/wrfout_d02_1980-02-09_03:00:00 timeidx is  33
/home/hasia/1979/wrfout_d02_1980-02-09_03:00:00 timeidx is  39
/home/hasia/1979/wrfout_d02_1980-02-14_03:00:00 timeidx is  1
/home/hasia/1979/wrfout_d02_1980-02-14_03:00:00 timeidx is  3
/home/hasia/1979/wrfout_d02_1980-02-14_03:00:00 timeidx is  5
/home/hasia/1979/wrfout_d02_1980-02-24_03:00:00 timeidx is  15
/home/hasia/1979/wrfout_d02_1980-02-24_03:00:00 timeidx is  17
/home/hasia/1979/wrfout_d02_1980-02-24_03:00:00 timeidx is  19
/home/hasia/1980/wrfout_d02_1981-01-20_03:00:00 timeidx is  29
/home/hasia/1980/wrfout_d02_1981-01-20_03:00:00 timeidx is  31
/home/hasia/1980/wrfout_d02_1981-01-20_03:00:00 timeidx is  33
/home/hasia/1980/wrfout_d02_1981-01-25_03:00:00 timeidx is  21
/home/hasia/1980/wrfout_d02_1981-01-25_03:00:00 timeidx is